## Item 17: Be Defensive When Iterating Over Arguments

* When a function takes a list of objects as a parameter, it's often important to iterate over that list multiple times.
* An iterator only produces its results a single time.
* If you iterate over an iterator or generator that has already raised a StopIteration exception, you won't get any results the second time around.

In [ ]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
visits = [15, 35, 80]

percentages = normalize(visits)
percentages

In [ ]:
# generator method

def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

* Problem 1

Surprisingly, calling normalize on the generator's return value produces no results.

In [ ]:
INPUT_TXT_FILE = "../data/my_numbers.txt"

it = read_visits(INPUT_TXT_FILE)
percentages = normalize(it)
print(percentages)

In [ ]:
it = read_visits(INPUT_TXT_FILE)
print(list(it))
print(list(it))  # already exhaused

In [ ]:
# explicityly exhaust an input iterator and 
# keep a copy of its entire contents in a list

def normalize_copy(numbers):
    numbers = list(numbers)  # copy the iterator
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
it = read_visits(INPUT_TXT_FILE)
percentages = normalize_copy(it)
print(percentages)

* Problem 2

    * The copy of the input iterator's contents could be large.

    * To get around this, accept a function (get_iter) that returns a new iterator each tine it's called.

    * To use normalize_func, you can pass in a lambda expression that calls the generator and produces a new iterator each time.

In [ ]:
def normalize_func(get_iter):
    total = sum(get_iter())  # new iterator
    result = []
    for value in get_iter():  # new iterator
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
percentages = normalize_func(lambda: read_visits(INPUT_TXT_FILE))

In [ ]:
percentages

* Problem 3
    * Though it works, having to pass a lambda function like this is clumsy.
    * The better way is to provide a new container class taht implements the iterator protocol.
    * `for x in foo` will actually call `iter(foo)`
    * The `iter` built-in function calls the `foo.__iter__` special method in return.
    * Implement the `__iter__` method as a generator.

In [ ]:
# define an iterable container class

class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [ ]:
visits = ReadVisits(INPUT_TXT_FILE)
percentages = normalize(visits)
percentages

* Problem 4
    * An iterator is passed to the `iter` built-in function, `iter` will return the iterator itself.
    * When a container type is passed to iter, a new iterator object will be returned each time.
    * Test an input value for this vehavior ad raise a `TypeError` to reject iterators.

In [ ]:
# ideal if don't want to copy the full input iterator
# also need to iterate over the input data multiple times.

def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):  # an iterator -- bad!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
normalize_defensive(visits)

In [ ]:
visits = ReadVisits(INPUT_TXT_FILE)
normalize_defensive(visits)

In [ ]:
it = iter(visits)
normalize_defensive(it)

### Things to Remember

* Beware of functions taht iterate over input arguments multiple times.
* Python's iterator protocol defines how `containers` and `iterators` interact with the `iter` and `next` built-in function.
* You can easily define your won iterable container type by implementing the `__iter__` method as a generator.
* You can detect atha a value is an iterator (instead of a container) if calling iter on it rwice productes the same result.
* Then progress with the `next` built-in function.